In [1]:
import pandas as pd

# Load datasets
patients = pd.read_csv(r"C:\Users\abhin\Downloads\Patient_Data (1).csv")
billing = pd.read_csv(r"C:\Users\abhin\Downloads\Billing_Data (1).csv")

# 1) Show info of patient dataset
print("=== Patient Dataset Info ===")
patients.info()

=== Patient Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [2]:
#Select billing-related columns
billing_relevant = patients[['PatientID','Department','Doctor','BillAmount']]
display(billing_relevant.head(10))

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN
5,101,Cardiology,Dr. Smith,5000.0


In [3]:
#Drop administrative columns
cleaned_patients = patients.drop(columns=[c for c in ['ReceptionistID','CheckInTime'] if c in patients.columns])
display(cleaned_patients.head(10))

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN
5,101,Alice,Cardiology,Dr. Smith,5000.0


In [4]:
# Department-wise total bill (groupby)
# Ensure BillAmount numeric then sum by Department
patients['BillAmount'] = pd.to_numeric(patients['BillAmount'], errors='coerce')
dept_total = patients.groupby('Department', dropna=False)['BillAmount'].sum().reset_index()
display(dept_total)


,Department,BillAmount
0,Cardiology,16200.0
1,Dermatology,0.0
2,Neurology,0.0
3,Orthopedics,7500.0


In [5]:
# Remove duplicate patients (based on PatientID)
unique_patients = patients.drop_duplicates(subset='PatientID').reset_index(drop=True)
display(unique_patients.head(10))
print("Unique patients shape:", unique_patients.shape)

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


Unique patients shape: (5, 7)


In [6]:
# Fill missing BillAmount with mean
mean_bill = patients['BillAmount'].mean()
patients['BillAmount'] = patients['BillAmount'].fillna(mean_bill)
print("Mean BillAmount used to fill missing values:", mean_bill)


Mean BillAmount used to fill missing values: 5925.0


In [7]:
# Merge billing dataset with patient dataset on PatientID
merged = pd.merge(patients, billing, on='PatientID', how='inner', suffixes=('_patient','_billing'))
display(merged.head(10))


,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00,2000,3000
1,102,Bob,Neurology,Dr. John,5925.0,2,2023-01-11 10:30,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00,3000,3200
4,105,Eva,Dermatology,Dr. Rose,5925.0,2,2023-01-14 08:45,1000,4000
5,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00,2000,3000


In [8]:
# Concatenate new patients (row-wise)
new_patients = pd.DataFrame({
    'PatientID':[9991,9992],
    'Department':['Dermatology','Cardiology'],
    'Doctor':['Dr. Nair','Dr. Rao'],
    'BillAmount':[3000.0,4500.0],
    'Diagnosis':['Skin Infection','Chest Pain']
})
updated_patients = pd.concat([patients.reset_index(drop=True), new_patients], axis=0, ignore_index=True, sort=False)
display(updated_patients.tail(10))


,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,Diagnosis
0,101,Alice,Cardiology,Dr. Smith,5000.0,1.0,2023-01-10 09:00,NaN
1,102,Bob,Neurology,Dr. John,5925.0,2.0,2023-01-11 10:30,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1.0,2023-01-12 11:00,NaN
3,104,David,Cardiology,Dr. Smith,6200.0,3.0,2023-01-13 12:00,NaN
4,105,Eva,Dermatology,Dr. Rose,5925.0,2.0,2023-01-14 08:45,NaN
5,101,Alice,Cardiology,Dr. Smith,5000.0,1.0,2023-01-10 09:00,NaN
6,9991,NaN,Dermatology,Dr. Nair,3000.0,NaN,NaN,Skin Infection
7,9992,NaN,Cardiology,Dr. Rao,4500.0,NaN,NaN,Chest Pain


In [9]:
# Concatenate new billing columns (column-wise)
new_cols = pd.DataFrame({
    'InsuranceCovered': [True] * len(updated_patients),
    'FinalAmount': updated_patients['BillAmount'] * 0.9
})
final_data = pd.concat([updated_patients.reset_index(drop=True), new_cols.reset_index(drop=True)], axis=1)
display(final_data.head(10))


,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,Diagnosis,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,1.0,2023-01-10 09:00,NaN,True,4500.0
1,102,Bob,Neurology,Dr. John,5925.0,2.0,2023-01-11 10:30,NaN,True,5332.5
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1.0,2023-01-12 11:00,NaN,True,6750.0
3,104,David,Cardiology,Dr. Smith,6200.0,3.0,2023-01-13 12:00,NaN,True,5580.0
4,105,Eva,Dermatology,Dr. Rose,5925.0,2.0,2023-01-14 08:45,NaN,True,5332.5
5,101,Alice,Cardiology,Dr. Smith,5000.0,1.0,2023-01-10 09:00,NaN,True,4500.0
6,9991,NaN,Dermatology,Dr. Nair,3000.0,NaN,NaN,Skin Infection,True,2700.0
7,9992,NaN,Cardiology,Dr. Rao,4500.0,NaN,NaN,Chest Pain,True,4050.0
